In [171]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)

import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'  

## EEG data and foof:

In [172]:
## data loading
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
foof = pd.read_csv(DATA_DIR+"/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/sensor_intercept_slope.csv")
foof

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [173]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Nov  2 11:45:24 2021',
 '__version__': '1.0',
 '__globals__': [],
 'x': array([[[7.70783437e-12, 1.51228585e-10, 2.47573539e-09, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.77038384e-11, 3.18378657e-10, 4.87969998e-09, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [7.40296713e-13, 2.31863417e-11, 8.15417955e-10, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [2.29975743e-06, 3.99951901e-06, 7.46542514e-06, ...,
          1.46918101e-04, 9.58585972e-05, 6.17663470e-05],
         [4.57044628e-04, 6.47727893e-04, 9.08127810e-04, ...,
          6.37886359e-06, 3.97923031e-06, 2.45564541e-06],
         [4.04096644e-03, 1.13127320e-02, 2.76608998e-02, ...,
          1.00828235e-11, 3.61710661e-12, 1.26965105e-12]],
 
        [[3.79871331e-02, 1.63943284e-01, 2.89759115e-01, ...,
          1.29540838e-01, 1.11913141e-01, 8.6

In [174]:
# flattening
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df = np.array(df)
df = df.reshape((2042,105,79)) 
df.shape

(2042, 105, 79)

In [175]:
df = pd.DataFrame(df.reshape((df.shape[0], -1)))
df.shape


(2042, 8295)

In [176]:

type(df)
df['IDs'] = foof['IDs']
df

,0,1,2,3,4,5,6,7,8,9,...,8286,8287,8288,8289,8290,8291,8292,8293,8294,IDs
0,7.707834e-12,1.512286e-10,2.475735e-09,3.453763e-08,4.554996e-07,0.000007,0.000109,0.001314,0.009992,0.045200,...,2.990881e-09,1.223086e-09,4.893694e-10,1.915752e-10,7.337775e-11,2.749867e-11,1.008282e-11,3.617107e-12,1.269651e-12,NDARAA075AMK
1,3.798713e-02,1.639433e-01,2.897591e-01,2.097609e-01,6.347556e-02,0.027758,0.105298,0.184513,0.130638,0.163258,...,5.835291e-06,1.856717e-06,5.922864e-07,1.928109e-07,6.449918e-08,2.204650e-08,7.598678e-09,2.604260e-09,8.780452e-10,NDARAA112DMH
2,2.104310e-06,4.299647e-05,5.644195e-04,4.760137e-03,2.579253e-02,0.089799,0.201005,0.290260,0.276177,0.196151,...,1.332064e-07,5.270957e-08,2.026971e-08,7.575282e-09,2.751336e-09,9.711398e-10,3.331297e-10,1.110552e-10,3.597966e-11,NDARAA117NEJ
3,1.692639e-01,2.021357e-01,2.273063e-01,2.406964e-01,2.400039e-01,0.225430,0.202278,0.198154,0.228356,0.181836,...,8.323825e-02,7.759521e-02,7.183415e-02,6.604061e-02,6.029416e-02,5.466678e-02,4.922161e-02,4.401211e-02,3.908162e-02,NDARAA947ZG5
4,8.332328e-06,1.552379e-04,1.804338e-03,1.287033e-02,5.615280e-02,0.149809,0.244887,0.247738,0.163687,0.094061,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,8.241315e-06,2.938160e-04,4.641777e-03,3.249546e-02,1.008094e-01,0.138615,0.084707,0.024339,0.009157,0.021386,...,5.852474e-03,4.262113e-03,3.068545e-03,2.184048e-03,1.536788e-03,1.069025e-03,7.351630e-04,4.998062e-04,3.359246e-04,NDARZN277NR6
2038,8.113400e-03,1.005675e-02,1.237930e-02,1.513279e-02,1.837070e-02,0.022147,0.026515,0.031525,0.037230,0.043721,...,5.855173e-04,3.933072e-04,2.601868e-04,1.695084e-04,1.087534e-04,6.871244e-05,4.275280e-05,2.619553e-05,1.580597e-05,NDARZN578YDP
2039,5.472767e-03,8.816325e-03,1.629356e-02,3.226822e-02,6.212373e-02,0.108594,0.166461,0.220594,0.252412,0.252807,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,2.658455e-03,5.749806e-03,1.180538e-02,2.300961e-02,4.257369e-02,0.074778,0.124685,0.197358,0.296551,0.423009,...,3.676327e-04,1.925477e-04,9.805386e-05,4.855040e-05,2.337344e-05,1.094093e-05,4.979521e-06,2.203549e-06,9.481114e-07,NDARZN677EYE


## Behavioral Data:


In [177]:
behaviour_data = pd.read_csv('/home/ubuntu/Martyna/repo/AI4Health/AllData.csv')
behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
behaviour_data.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [178]:
behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data.shape

(2579, 9)

In [179]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 9 columns (with column 'IDs').
There are 8 columns that have missing values.


,Missing Values,% of Total Values
WISC_WMI_Sum,712,27.6
WISC_VCI_Sum,702,27.2
SCARED_P_GD,576,22.3
SRS_SCI_T,448,17.4
SRS_RRB_T,447,17.3
SWAN_IN_Avg,385,14.9
SWAN_HY_Avg,385,14.9
SWAN_Avg,385,14.9


## Merge EEG and foof data together:

In [180]:
data = pd.merge(df, foof, on='IDs', how='inner')
data.shape

(2042, 8506)

## and then with the behaviorals

In [181]:
df = pd.merge(data, behaviour_data, on='IDs', how='inner')

df.shape

(1465, 8514)

## features and labels preparation:

In [182]:
#removing NaNs
df = df.dropna()
df.shape

(1075, 8514)

In [183]:
df = df.drop(columns=['IDs'])
df.shape

(1075, 8513)

In [184]:
X = df.iloc[:, 0:8731].to_numpy()
X.shape

(1075, 8513)

In [185]:
Y = df[[ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]

## TARGETS NORMALISATION
Y = Y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
Y_scaled = min_max_scaler.fit_transform(Y)
Y = pd.DataFrame(Y_scaled)
print(Y.shape)
Y.mean()

(1075, 8)


0    0.412300
1    0.330517
2    0.618501
3    0.542498
4    0.580500
5    0.260155
6    0.456076
7    0.494200
dtype: float64

## PCA on X dimension



In [186]:
# dimensionality reduction
pca = PCA(.90) # 95% variance retained
pca.fit(X)
# transform data
X_pca = pca.transform(X)
X_pca.shape

X = X_pca
print("X:", X.shape, "Y:", Y.shape)


X: (1075, 14) Y: (1075, 8)


In [187]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.25)

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [188]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor



xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.5)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))

print(ytest[:10])
print(Y.mean())

Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.13
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.03
             0         1         2         3         4         5        6  \
585   0.722222  0.551020  0.759259  0.833333  0.796296  0.222222  0.37500   
935   0.388889  0.244898  0.685185  0.518518  0.601852  0.222222  0.37500   
1071  0.259259  0.265306  0.629630  0.500000  0.564815  0.055556  0.46875   
97    0.925926  0.795918  0.925926  0.648148  0.787037  0.500000  0.31250   
836   0.462963  0.326531  0.407408  0.555555  0.481482  0.444444  0.46875   
709   0.518519  0.428571  0.722222  0.388889  0.555555  0.722222  0.34375   
1001  0.259259  0.285714  0.592592  0.611111  0.601852  0.388889  0.50000   
348   0.537037  0.367347  0.666667  0.462963  0.564815  0.444444  0.34375   
1058  0.055556  0.183673  0.796296  0.537037  0.666667  0.111111  0.28125   
118   0.907407  0.897959  0.851852  0.555555  0.703704  0.277778  0.34375   

             7  
585   

## SVR regressor

In [189]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (SVC): {:.2f}".format(mor.score(xtest, ytest)))


Mean squared error (SVC): 0.02
R2 score (SVC): 0.49


## Linear Regression

In [190]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

# Generate predictions for testing data
ypred = model.predict(xtest)

print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))												



Mean squared error (LinReg): 0.01
R2 score (LinReg): 0.71


## Random forest regressor

In [191]:
max_depth = 30
regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
regr_rf.fit(xtrain, ytrain)
# Predict on new data
y_rf = regr_rf.predict(xtest)
regr_rf.score(xtest, ytest)



print("Mean squared error (RandomForest): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (RandomForest): {:.2f}".format(regr_rf.score(xtest, ytest)))


Mean squared error (RandomForest): 0.01
R2 score (RandomForest): 0.55


 ## multi-output meta estimator

In [192]:
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
regr_multirf.fit(xtrain, ytrain)
y_multirf = regr_multirf.predict(xtest)


print("Mean squared error (regr_multirf): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (regr_multirf): {:.2f}".format(regr_multirf.score(xtest, ytest)))

Mean squared error (regr_multirf): 0.01
R2 score (regr_multirf): 0.63


## regressor chains

In [193]:
# example of evaluating chained multioutput regression with an SVM model
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define base model
model = LinearSVR()
scoring = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"] 
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = model_selection.cross_validate(wrapper, X, Y, scoring=scoring, cv=cv, n_jobs=-1)


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai

In [194]:
for i in n_scores:
    print(i)
    print('results: %.3f (%.3f)' % (mean(n_scores[i]), std(n_scores[i])))

fit_time
results: 0.475 (0.016)
score_time
results: 0.004 (0.000)
test_r2
results: 0.438 (0.242)
test_neg_mean_squared_error
results: -0.019 (0.007)
test_neg_mean_absolute_error
results: -0.066 (0.012)


## mlp with keras

In [195]:
model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")


model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score= np.array(score)
score.mean()

Epoch 1/100
36/36 [==============================] - 0s 3ms/step - loss: 2.3982 - mse: 2.3982 - val_loss: 0.6468 - val_mse: 0.6468
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3653 - mse: 0.3653 - val_loss: 0.3101 - val_mse: 0.3101
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2035 - mse: 0.2035 - val_loss: 0.2120 - val_mse: 0.2120
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.1400 - mse: 0.1400 - val_loss: 0.1660 - val_mse: 0.1660
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.1081 - mse: 0.1081 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 6/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0873 - mse: 0.0873 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0727 - mse: 0.0727 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 8/100
36/36 [==============================] - 0s 2ms/step - loss: 0.0627 - m

0.02792408